## 3-1 표본추출

In [1]:
# plotting 환경 설정
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (5,5) # 그래프 크기
plt.rcParams['font.family'] = 'Malgun Gothic' # 글꼴
plt.rcParams['font.size'] = 12 # 글꼴 크기
plt.rcParams['axes.unicode_minus'] = False # 마이너스 표기

단순랜덤추출법
+ 각 샘플에 번호를 부여하여 임의의 n개를 추출하는 방법

In [10]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_iris

data = load_iris()
iris_columns = list(data['feature_names'] + ['target'])
iris = pd.DataFrame(np.c_[data['data'], data['target']], columns= [col.replace(' (cm)', '') for col in iris_columns])
iris.head()

,sepal length,sepal width,petal length,petal width,target
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0


In [19]:
# 복원하지 않고 3개를 랜덤추출
iris.sample(n = 3, replace = False, random_state= 42)

,sepal length,sepal width,petal length,petal width,target
73,6.1,2.8,4.7,1.2,1.0
18,5.7,3.8,1.7,0.3,0.0
118,7.7,2.6,6.9,2.3,2.0


In [18]:
# frac 파라미터로 전체 데이터의 3%를 랜덤추출
iris.sample(frac = 0.03, random_state= 42)

,sepal length,sepal width,petal length,petal width,target
73,6.1,2.8,4.7,1.2,1.0
18,5.7,3.8,1.7,0.3,0.0
118,7.7,2.6,6.9,2.3,2.0
78,6.0,2.9,4.5,1.5,1.0


In [26]:
# 리스트에서 단순랜덤추출
import random

data_list = [1,2,3,4,5, 'a', 'b', 'c']
print('random.sample :', random.sample(data_list, 4))
print('np.random.choice :', np.random.choice(data_list, 4, replace= True))

random.sample : ['a', 5, 'c', 1]
np.random.choice : ['4' '2' '2' 'b']


In [30]:
# [참고] 난수 생성
print(np.random.randint(0, 10, 3))
print(np.random.rand(2,3))

[0 2 8]
[[0.0466921  0.54865268 0.57543522]
 [0.90200326 0.65774118 0.67795742]]


계통추출법
+ 단순랜덤추출법의 변형된 방식
+ N개 모집단에서 n개 추출할 때, N/n으로 구간을 나누고 그 구간에 있는 K개의 샘플 중에 임의로 하나 선택한다.

In [46]:
# 계통추출법으로 표본 추출
data, n = iris, 8

N = len(data)
K = N // n

idx = data[:K].sample(1).index

sys_df = pd.DataFrame()
while len(sys_df) < n:
    sys_df = pd.concat([sys_df, data.loc[idx, :]], axis = 0)
    idx += K

print('N : {}, n : {}, K : {}'.format(N, n, K))

sys_df

N : 150, n : 8, K : 18


,sepal length,sepal width,petal length,petal width,target
11,4.8,3.4,1.6,0.2,0.0
29,4.7,3.2,1.6,0.2,0.0
47,4.6,3.2,1.4,0.2,0.0
65,6.7,3.1,4.4,1.4,1.0
83,6.0,2.7,5.1,1.6,1.0
101,5.8,2.7,5.1,1.9,2.0
119,6.0,2.2,5.0,1.5,2.0
137,6.4,3.1,5.5,1.8,2.0


집락추출법
+ 군집별로 랜덤추출법을 수행하여 포본을 얻는 방법
+ 지역표본추출과 다단계표본추출이 이에 해당
+ 군집과 군집은 비교적 유사한 특징을 띈다.

층화추출법
+ 계층별로 랜덤추출법을 수행하여 표본을 얻는 방법
+ 각 층 내 요소들은 유사하지만 층과 층의 요소들은 상이하다.

In [48]:
# target 을 층 혹은 집락이라고 가정
# 원본 데이터 분포 확인

iris['target'].value_counts()

target
0.0    50
1.0    50
2.0    50
Name: count, dtype: int64

In [65]:
# 데이터, 층/집락 정보를 가진 컬럼명, 추출 표본 개수
sampling_num = 9

# 비례층화추출법 : 원본 데이터의 비율대로 추출
levels = iris['target'].unique()
prop_val = iris['target'].value_counts(normalize= True)
num = prop_val * sampling_num
print(num)

result = pd.DataFrame()

for level in levels:
    temp_df = iris[iris['target'] == level].sample(int(num[level]))
    result = pd.concat([result, temp_df], axis = 0)

print(result)

target
0.0    3.0
1.0    3.0
2.0    3.0
Name: proportion, dtype: float64
     sepal length  sepal width  petal length  petal width  target
14            5.8          4.0           1.2          0.2     0.0
44            5.1          3.8           1.9          0.4     0.0
25            5.0          3.0           1.6          0.2     0.0
72            6.3          2.5           4.9          1.5     1.0
51            6.4          3.2           4.5          1.5     1.0
84            5.4          3.0           4.5          1.5     1.0
141           6.9          3.1           5.1          2.3     2.0
132           6.4          2.8           5.6          2.2     2.0
136           6.3          3.4           5.6          2.4     2.0


In [69]:
# 불비례층화추출법 : 임의로 정한 특정 비율대로 샘플링
sampling_num, proportion = 10, {0:0.2, 1:0.5, 2:0.3}

levels = list(proportion.keys())
prop_val = np.array(list(proportion.values()))
num = prop_val * sampling_num

result = pd.DataFrame()
for level in levels:
    temp_df = iris[iris['target'] == level].sample(int(num[level]))
    result = pd.concat([result, temp_df], axis = 0)

result

,sepal length,sepal width,petal length,petal width,target
17,5.1,3.5,1.4,0.3,0.0
33,5.5,4.2,1.4,0.2,0.0
96,5.7,2.9,4.2,1.3,1.0
69,5.6,2.5,3.9,1.1,1.0
91,6.1,3.0,4.6,1.4,1.0
50,7.0,3.2,4.7,1.4,1.0
84,5.4,3.0,4.5,1.5,1.0
130,7.4,2.8,6.1,1.9,2.0
126,6.2,2.8,4.8,1.8,2.0
120,6.9,3.2,5.7,2.3,2.0


## 3-2 데이터분할
+ 과적합을 피하기 위함

일반적 데이터 분할(7:3) 및 홀드아웃 방법(5:5)

In [73]:
from sklearn.model_selection import train_test_split
X = iris.drop('target', axis=1)
y = iris['target']

# 일반적 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.3, random_state= 42)
print(X_train.shape, X_test.shape)

# 홀드아웃
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.5, random_state= 42)
print(X_train.shape, X_test.shape)

(105, 4) (45, 4)
(75, 4) (75, 4)


shuffle split
+ 무작위 순열 교차 검증에 사용

In [92]:
from sklearn.model_selection import ShuffleSplit
from collections import Counter

# n_split : fold개수, shuffle : 데이터 분할 전 shuffle 여부
kf = ShuffleSplit(n_splits= 4, test_size= 0.25)

for i, (train_index, test_index) in enumerate(kf.split(X)):
    print('sample {} => train_index : {}, test_index : {}'.format(i, train_index[:3], test_index[:3]))
    X_train, X_test, y_train, y_test = X.iloc[train_index, :], X.iloc[test_index, :], y.iloc[train_index], y.iloc[test_index]
    print('\tX_train의 비율 : {:.2f}, X_test의 비율 : {:.2f}'.format(len(X_train)/len(X), len(X_test)/ len(X)))
    print('\ty_train의 타겟구성 :', Counter(y_train))
    print('\ty_test의 타겟구성 :', Counter(y_test), '\n')

sample 0 => train_index : [ 77 112  43], test_index : [ 31 103   3]
	X_train의 비율 : 0.75, X_test의 비율 : 0.25
	y_train의 타겟구성 : Counter({2.0: 41, 1.0: 37, 0.0: 34})
	y_test의 타겟구성 : Counter({0.0: 16, 1.0: 13, 2.0: 9}) 

sample 1 => train_index : [45 55 87], test_index : [ 85 149 102]
	X_train의 비율 : 0.75, X_test의 비율 : 0.25
	y_train의 타겟구성 : Counter({0.0: 41, 2.0: 36, 1.0: 35})
	y_test의 타겟구성 : Counter({1.0: 15, 2.0: 14, 0.0: 9}) 

sample 2 => train_index : [39  6 56], test_index : [ 90 127 121]
	X_train의 비율 : 0.75, X_test의 비율 : 0.25
	y_train의 타겟구성 : Counter({1.0: 39, 0.0: 38, 2.0: 35})
	y_test의 타겟구성 : Counter({2.0: 15, 0.0: 12, 1.0: 11}) 

sample 3 => train_index : [ 68 111 114], test_index : [ 44 133  20]
	X_train의 비율 : 0.75, X_test의 비율 : 0.25
	y_train의 타겟구성 : Counter({2.0: 38, 1.0: 37, 0.0: 37})
	y_test의 타겟구성 : Counter({0.0: 13, 1.0: 13, 2.0: 12}) 



K-fold 분할
+ K개의 집단으로 분할하고 K-1, 1 를 학습용, 검증용으로 삼는 방법

In [87]:
from sklearn.model_selection import KFold
from collections import Counter

# n_split : fold개수, shuffle : 데이터 분할 전 shuffle 여부
kf = KFold(n_splits= 4, shuffle= False)

for i, (train_index, test_index) in enumerate(kf.split(X)):
    print('sample {} => train_index : {}, test_index : {}'.format(i, train_index[:3], test_index[:3]))
    X_train, X_test, y_train, y_test = X.iloc[train_index, :], X.iloc[test_index, :], y.iloc[train_index], y.iloc[test_index]
    print('\tX_train의 비율 : {:.2f}, X_test의 비율 : {:.2f}'.format(len(X_train)/len(X), len(X_test)/ len(X)))
    print('\ty_train의 타겟구성 :', Counter(y_train))
    print('\ty_test의 타겟구성 :', Counter(y_test))

sample 0 => train_index : [38 39 40], test_index : [0 1 2]
	X_train의 비율 : 0.75, X_test의 비율 : 0.25
	y_train의 타겟구성 : Counter({1.0: 50, 2.0: 50, 0.0: 12})
	y_test의 타겟구성 : Counter({0.0: 38})
sample 1 => train_index : [0 1 2], test_index : [38 39 40]
	X_train의 비율 : 0.75, X_test의 비율 : 0.25
	y_train의 타겟구성 : Counter({2.0: 50, 0.0: 38, 1.0: 24})
	y_test의 타겟구성 : Counter({1.0: 26, 0.0: 12})
sample 2 => train_index : [0 1 2], test_index : [76 77 78]
	X_train의 비율 : 0.75, X_test의 비율 : 0.25
	y_train의 타겟구성 : Counter({0.0: 50, 2.0: 37, 1.0: 26})
	y_test의 타겟구성 : Counter({1.0: 24, 2.0: 13})
sample 3 => train_index : [0 1 2], test_index : [113 114 115]
	X_train의 비율 : 0.75, X_test의 비율 : 0.25
	y_train의 타겟구성 : Counter({0.0: 50, 1.0: 50, 2.0: 13})
	y_test의 타겟구성 : Counter({2.0: 37})


Stratified K-fold 분할
+ k-fold 인데 fold별로 클래스들의 비율이 모두 같도록

In [93]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits= 4)

# 분할 시 y를 고려해야 하므로 split에 y를 입력
for i, (train_index, test_index) in enumerate(skf.split(X, y)):
    print('sample {} => train_index : {}, test_index : {}'.format(i, train_index[:3], test_index[:3]))
    X_train, X_test, y_train, y_test = X.iloc[train_index, :], X.iloc[test_index, :], y.iloc[train_index], y.iloc[test_index]
    print('\tX_train의 비율 : {:.2f}, X_test의 비율 : {:.2f}'.format(len(X_train)/len(X), len(X_test)/ len(X)))
    print('\ty_train의 타겟구성 :', Counter(y_train))
    print('\ty_test의 타겟구성 :', Counter(y_test))

sample 0 => train_index : [13 14 15], test_index : [0 1 2]
	X_train의 비율 : 0.75, X_test의 비율 : 0.25
	y_train의 타겟구성 : Counter({1.0: 38, 0.0: 37, 2.0: 37})
	y_test의 타겟구성 : Counter({0.0: 13, 2.0: 13, 1.0: 12})
sample 1 => train_index : [0 1 2], test_index : [13 14 15]
	X_train의 비율 : 0.75, X_test의 비율 : 0.25
	y_train의 타겟구성 : Counter({1.0: 38, 0.0: 37, 2.0: 37})
	y_test의 타겟구성 : Counter({0.0: 13, 2.0: 13, 1.0: 12})
sample 2 => train_index : [0 1 2], test_index : [26 27 28]
	X_train의 비율 : 0.75, X_test의 비율 : 0.25
	y_train의 타겟구성 : Counter({0.0: 38, 2.0: 38, 1.0: 37})
	y_test의 타겟구성 : Counter({1.0: 13, 0.0: 12, 2.0: 12})
sample 3 => train_index : [0 1 2], test_index : [38 39 40]
	X_train의 비율 : 0.75, X_test의 비율 : 0.25
	y_train의 타겟구성 : Counter({0.0: 38, 2.0: 38, 1.0: 37})
	y_test의 타겟구성 : Counter({1.0: 13, 0.0: 12, 2.0: 12})


## 3-3 교차 검증

분할 샘플들로 교차 검증

In [95]:
import warnings
warnings.filterwarnings(action= 'ignore')

In [96]:
from sklearn.model_selection import cross_validate, StratifiedKFold
from sklearn.linear_model import LogisticRegression

X = iris.drop('target', axis = 1)
y = iris['target']

lr = LogisticRegression()
SKF = StratifiedKFold(n_splits=4)
result = cross_validate(lr, X, y, cv = SKF, return_train_score=True) # 교차검증
print(pd.DataFrame(result))


   fit_time  score_time  test_score  train_score
0  0.019461    0.001988    0.973684     0.964286
1  0.008093    0.001933    0.973684     0.973214
2  0.008022    0.001929    0.945946     0.982301
3  0.008309    0.000964    1.000000     0.964602


GridsearchCV 로 교차 검증

In [102]:
from sklearn.model_selection import GridSearchCV

lr = LogisticRegression()
param_grid = {'C' : [0.01, 0.1, 1], 'solver' : ['lbfgs', 'liblinear'], 'max_iter' : [100, 200, 300]}
SKF = StratifiedKFold(n_splits=4)

lr_grid = GridSearchCV(lr, param_grid=param_grid, cv = SKF)
lr_grid.fit(X, y)

print(lr_grid.best_score_)
print(lr_grid.best_params_)
print(pd.DataFrame(lr_grid.cv_results_))

0.9733285917496444
{'C': 1, 'max_iter': 100, 'solver': 'lbfgs'}
    mean_fit_time  std_fit_time  mean_score_time  std_score_time param_C  \
0        0.006520      0.002325         0.002021        0.000707    0.01   
1        0.003000      0.000706         0.001752        0.000433    0.01   
2        0.009463      0.004254         0.001635        0.000661    0.01   
3        0.001999      0.000708         0.001251        0.000434    0.01   
4        0.001541      0.001672         0.000250        0.000433    0.01   
5        0.000751      0.000830         0.000405        0.000425    0.01   
6        0.008215      0.008235         0.000000        0.000000     0.1   
7        0.000000      0.000000         0.000000        0.000000     0.1   
8        0.007819      0.006940         0.000250        0.000433     0.1   
9        0.003392      0.005875         0.000276        0.000479     0.1   
10       0.004935      0.006443         0.000000        0.000000     0.1   
11       0.000000      0

## 연습문제

1. 랜덤포레스트 알고리즘의 파라미터 max_depth 의 후보 4개를 토대로 교차분석을 진행하고 그 결과를 표로 나타내시오

In [103]:
df = pd.read_csv('./data/practice1_bank.csv')
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,month,y
0,30,unemployed,married,primary,no,1787,no,no,cellular,oct,no
1,33,services,married,secondary,no,4789,yes,yes,cellular,may,no
2,35,management,single,tertiary,no,1350,yes,no,cellular,apr,no
3,30,management,married,tertiary,no,1476,yes,yes,unknown,jun,no
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,may,no


In [111]:
# 범주형 변수들을 레이블 인코딩을 통해 전처리
df2 = df.copy()
for col in [i for i in df.columns if df[i].dtypes == object]:
    DICT = dict(zip(df[col].unique(), [i for i in range(df[col].nunique())]))
    print(DICT, '\n')
    df2[col] = df2[col].map(DICT)

print(df2.head())

{'unemployed': 0, 'services': 1, 'management': 2, 'blue-collar': 3, 'self-employed': 4, 'technician': 5, 'entrepreneur': 6, 'admin.': 7, 'student': 8, 'housemaid': 9, 'retired': 10, 'unknown': 11} 

{'married': 0, 'single': 1, 'divorced': 2} 

{'primary': 0, 'secondary': 1, 'tertiary': 2, 'unknown': 3} 

{'no': 0, 'yes': 1} 

{'no': 0, 'yes': 1} 

{'no': 0, 'yes': 1} 

{'cellular': 0, 'unknown': 1, 'telephone': 2} 

{'oct': 0, 'may': 1, 'apr': 2, 'jun': 3, 'feb': 4, 'aug': 5, 'jan': 6, 'jul': 7, 'nov': 8, 'sep': 9, 'mar': 10, 'dec': 11} 

{'no': 0, 'yes': 1} 

   age  job  marital  education  default  balance  housing  loan  contact  \
0   30    0        0          0        0     1787        0     0        0   
1   33    1        0          1        0     4789        1     1        0   
2   35    2        1          2        0     1350        1     0        0   
3   30    2        0          2        0     1476        1     1        1   
4   59    3        0          1        0        

In [112]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold

X = df2.drop('y', axis= 1)
y = df2['y']

param_grid = {'max_depth' : [3,4,5,6]}
rf_clf = RandomForestClassifier()
SKF = StratifiedKFold(n_splits=5) # train_test_split(x,y, stratify = y, test_size = 0.2) 를 5번 시행한 것
grid = GridSearchCV(rf_clf, param_grid= param_grid, cv = SKF)
grid.fit(X, y)

print(grid.best_params_)
print(grid.best_score_)
print(pd.DataFrame(grid.cv_results_))

{'max_depth': 5}
0.8849811763555469
   mean_fit_time  std_fit_time  mean_score_time  std_score_time  \
0       0.171997      0.028823         0.010274        0.003261   
1       0.179385      0.005825         0.010914        0.002318   
2       0.194614      0.002979         0.006655        0.003145   
3       0.218488      0.010164         0.010877        0.003340   

  param_max_depth            params  split0_test_score  split1_test_score  \
0               3  {'max_depth': 3}           0.883978           0.884956   
1               4  {'max_depth': 4}           0.883978           0.884956   
2               5  {'max_depth': 5}           0.885083           0.884956   
3               6  {'max_depth': 6}           0.885083           0.884956   

   split2_test_score  split3_test_score  split4_test_score  mean_test_score  \
0           0.884956           0.884956           0.884956         0.884760   
1           0.884956           0.884956           0.884956         0.884760   
2    